## Estimating roadside leafiness from satellite data

Author: Robin Wilson

Email: robin@rtwilson.com

This is designed to be run on Microsoft Planetary Computer. Most code should work locally, but will require a bit of modification (particularly the Dask cluster setup)

In [27]:
import numpy as np
import xarray as xr

import rasterio.features
import stackstac
import pystac_client
import planetary_computer
import rioxarray

import osmnx
import geopandas as gpd
import rasterio as rio
from rasterio import features
from rasterstats import zonal_stats
import pandas as pd

import xrspatial.multispectral as ms

from dask_gateway import GatewayCluster

### Create the Dask cluster
Dask Gateway is set up on Planetary Computer, so all we need to do here is connect to it, and say we want a cluster with a minimum of 4 notes, and a maximum of 24

In [2]:
cluster = GatewayCluster()

client = cluster.get_client()

cluster.adapt(minimum=4, maximum=24)
print(cluster.dashboard_link)

https://pccompute.westeurope.cloudapp.azure.com/compute/services/dask-gateway/clusters/prod.5cb89d16b9b54101b801dc3ffaacec29/status


### Set the Area of Interest (AOI)
Go to https://geojson.io/ and draw a polygon, and copy the GeoJSON here

In [28]:
area_of_interest = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          [
            -1.5479076771483165,
            50.96242621046028
          ],
          [
            -1.2804063094592664,
            50.96242621046028
          ],
          [
            -1.2794838909493649,
            50.83618532389028
          ],
          [
            -1.4593555002576863,
            50.85249370448824
          ],
          [
            -1.5442180031104726,
            50.911854605422235
          ],
          [
            -1.5479076771483165,
            50.96242621046028
          ]
        ],
        "type": "LineString"
      }
    }
  ]
}

### Search the STAC Catalog
The Spatio-Temporal Asset Catalog is a specification for catalogues of satellite data.

In [ ]:
# Get the bounding box (rectangle that covers the polygon) of the polygon
bbox = rasterio.features.bounds(area_of_interest)

In [30]:
# Connect to the STAC API
stac = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

# Run the search over the bounding box, for Sentinel 2 L2A imagery (a particular satellite, operating in the optical bands,
# run by the European Space Agency), over a date range of 1st Jan 2022 to 1st Jan 2023, with cloud cover less than 10%
search = stac.search(
    bbox=bbox,
    datetime="2022-01-01/2023-01-01",
    collections=["sentinel-2-l2a"],
    query={"eo:cloud_cover": {"lt": 10}},
)

# Get the items and print the count
items = search.item_collection()
print(len(items))

37


### Combine the satellite data into a big multi-dimensional array

In [34]:
# Stack the data together. This combines all of the satellite images into a big multi-dimensional array
# Because the array is an XArray DataArray, it has nice labels - so we know which dimension is x, y, band, time etc
# Here we're using B08 and B04 which are the Near-Infrared and Red bands from this satellite
data = (
    stackstac.stack(
        items,
        assets=["B08", "B04"],
        chunksize=4096,
        resolution=10,
        bounds_latlon=bbox
    )
    .where(lambda x: x > 0, other=np.nan)  # sentinel-2 uses 0 as nodata, so set it to NaN
    .assign_coords(band=lambda x: x.common_name.rename("band"))  # use common names for the bands, to make it easier
)
# Just printing out the variable provides a nice Jupyter representation showing a nice diagram of the array
data

/srv/conda/envs/notebook/lib/python3.11/site-packages/stackstac/prepare.py:363: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  times = pd.to_datetime(


<xarray.DataArray 'stackstac-92c6346394ee6992c42ef9675c535d8d' (time: 37,
                                                                band: 2,
                                                                y: 1445, x: 1918)>
dask.array<where, shape=(37, 2, 1445, 1918), dtype=float64, chunksize=(1, 1, 1445, 1918), chunktype=numpy.ndarray>
Coordinates: (12/44)
  * time                                     (time) datetime64[ns] 2022-01-14...
    id                                       (time) <U54 'S2B_MSIL2A_20220114...
  * band                                     (band) <U3 'nir' 'red'
  * x                                        (x) float64 6.02e+05 ... 6.211e+05
  * y                                        (y) float64 5.647e+06 ... 5.633e+06
    s2:water_percentage                      (time) float64 2.348 ... 19.53
    ...                                       ...
    gsd                                      float64 10.0
    proj:shape                               object {10980}
    common_name                              (band) <U3 'nir' 'red'
    center_wavelength                        (band) float64 0.842 0.665
    full_width_half_max                      (band) float64 0.145 0.038
    epsg                                     int64 32630
Attributes:
    spec:        RasterSpec(epsg=32630, bounds=(601970, 5632610, 621150, 5647...
    crs:         epsg:32630
    transform:   | 10.00, 0.00, 601970.00|\n| 0.00,-10.00, 5647060.00|\n| 0.0...
    resolution:  10

In [35]:
# Calculate the median over time (going through the stack of satellite images, calculating a median over time for each
# pixel)
%%time
median = data.median(dim="time").compute()

CPU times: user 124 ms, sys: 88.1 ms, total: 212 ms
Wall time: 13.8 s


### Calculate NDVI

In [38]:
# Extract the Near-Infrared and Red bands
nir = median.sel(band='nir')
red = median.sel(band='red')

In [39]:
# Calculate the Normalised Difference Vegetation Index (NDVI) - a measure of vegetation greenness
ndvi = (nir - red)/(nir + red)

In [40]:
# Save it out to a TIFF file
ndvi.rio.to_raster("S2_Median_2022_AOI.tif")

### Get roads from OSM

In [ ]:
# This takes a parameter for (latitude, longitude) and a radius (dist) and gives back all the roads
roads = osmnx.graph_from_point((50.91404,-1.41640), dist=10000, network_type="drive")

KeyboardInterrupt: 

In [ ]:
osmnx.save_graph_geopackage(roads, "SotonRoads.gpkg")

### Buffer roads

In [ ]:
roads = gpd.read_file("SotonRoads.gpkg", layer='edges')

In [ ]:
roads.geometry = roads.geometry.buffer(10)

/var/folders/v0/p9_n6wqj1jb7xn4q18f7p_q40000gn/T/ipykernel_58610/2057971081.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  roads.geometry = roads.geometry.buffer(10)
/Users/robin/mambaforge/envs/fibreplanner/lib/python3.11/site-packages/pygeos/constructive.py:175: RuntimeWarning: invalid value encountered in buffer
  return lib.buffer(


### Rasterize the roads
Here we want to convert the roads from a vector dataset to a raster, so we can easily use it to mask the satellite imagery data

In [ ]:
def rasterize_roads(roads_gdf, aerial_filename, output_filename):
    """
    Rasterize roads data from the given GDF into a raster with the same configuration as a given aerial image raster.

    Parameters:
    - roads_gdf: A GeoDataFrame containing the road lines to be rasterized
    - aerial_filename: A raster file from which the configuration (size, resolution, geospatial reference) will be taken
                       for the rasterized output.
    - output_filename: The filename to save the output to.
    """
    with rio.open(aerial_filename) as f:
        aerial_metadata = f.meta

    aerial_metadata['crs'] = 32630
    roads_reproj = roads_gdf.to_crs(aerial_metadata["crs"])

    meta = aerial_metadata.copy()
    meta["count"] = 1
    del meta["nodata"]
    with rio.open(output_filename, "w+", **meta) as out:
        out_arr = out.read(1)

        # this is where we create a generator of geom, value pairs to use in rasterizing
        # we're setting all values to 1, so we just hardcode that
        shapes = ((geom, 1) for geom in roads_reproj.geometry)

        # This is the main part, that rasterises the shapes, filling empty cells with 0
        # The transform is the bit that holds the metadata like pixel size, top-left corner co-ordinate,
        # crs etc
        burned = features.rasterize(
            shapes=shapes, fill=0, out=out_arr, transform=out.transform
        )
        print(f"Total burned pixels with value of 1: {burned.sum()}")
        out.write_band(1, burned)

In [ ]:
rasterize_roads(roads, "S2_Median_2022_AOI.tif", "AOI_Roads.tif")

{'driver': 'GTiff', 'dtype': 'float64', 'nodata': None, 'width': 1918, 'height': 1445, 'count': 1, 'crs': 32630, 'transform': Affine(10.0, 0.0, 601965.0,
       0.0, -10.0, 5647065.0)}
Total burned pixels with value of 1: 388360.0


# Load data and mask

In [ ]:
# Loads the 1st band from the GeoTIFF file, using rasterio
with rio.open("AOI_Roads.tif") as f:
    roads_raster = f.read(1)
    # Extract the metadata (transform, size, crs etc) too
    meta = f.meta

In [ ]:
with rio.open("S2_Median_2022_AOI.tif") as f:
    ndvi_raster = f.read(1)

In [ ]:
# Do the actual masking
# 1 x anything = anything
# 0 x anything = 0
masked = roads_raster * ndvi_raster

In [ ]:
# Write out to a GeoTIFF file (using the metadata we extracted from the input)
with rio.open("Masked_NDVI.tif", 'w', **meta) as f:
    f.write(masked, 1)

# Load wards data

In [ ]:
wards = gpd.read_file("SotonWards.shp")

In [ ]:
# Convert to the same co-ordinate system as the satellite data
wards = wards.to_crs(meta['crs'])

In [ ]:
wards.explore()

### Calculate zonal statistics

In [ ]:
# The params say which raster to use, that we want to calculate mean, and (importantly) that 0 is a no data value
# and shouldn't be taken into account in the statistics
roadside_stats = zonal_stats(
        vectors=wards.geometry, 
        raster='Masked_NDVI.tif', 
        stats='mean',
        nodata=0,
        prefix="roadside_"
    )

In [ ]:
# Join the zonal stats output back to the main dataframe as an extra column
wards = wards.join(pd.DataFrame(roadside_stats), how='left')

In [ ]:
# Produce a choropleth map where the polygons are coloured according to the value in this column
wards.explore('roadside_mean')

/Users/robin/mambaforge/envs/fibreplanner/lib/python3.11/site-packages/geopandas/explore.py:400: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif pd.api.types.is_categorical_dtype(gdf[column]):


In [ ]:
# Look at our data
wards[['wd17cd', 'wd17nm', 'roadside_mean']]

,wd17cd,wd17nm,roadside_mean,overall_mean
0,E05002455,Bargate,0.151769,0.152316
1,E05002456,Bassett,0.297117,0.383503
2,E05002457,Bevois,0.138306,0.148619
3,E05002458,Bitterne,0.250498,0.304511
4,E05002459,Bitterne Park,0.212127,0.312092
5,E05002460,Coxford,0.252240,0.323646
6,E05002461,Freemantle,0.190042,0.179213
7,E05002462,Harefield,0.254662,0.314489
8,E05002463,Millbrook,0.183902,0.170176
9,E05002464,Peartree,0.195554,0.263064
